In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 6
BATCH_SIZE = 1

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [ ]:
from vnet import VNet

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs

#checkpoint_refine = torch.load('../refine_bv_vnet2_save/2019-08-19 00:10:52.181257 epoch: 60.pth')
    
refine_model = VNet(classnum=1, in_channels=1)
refine_model = nn.DataParallel(refine_model)
refine_model = convert_model(refine_model)

#refine_model.load_state_dict(checkpoint_refine['state_dict_1'])

refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-2)
#optimizer.load_state_dict(checkpoint_refine['optimizer'])

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
#scheduler.load_state_dict(checkpoint_refine['scheduler'])

#epoch = checkpoint_refine['epoch']
epoch = 0
print(epoch)

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

0


In [ ]:
epochs = 5000

record = open('train_bv_refine_vnet2.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        train_losses=[]
        for minibatch in range(BATCH_SIZE):
            torch.cuda.empty_cache()
            
            refine_model.train()
            deeplab.eval()
            # Set the model flag to train
            # 1. enable dropout
            # 2. batchnorm behave differently in train and test
            #print(batch['image1_data'])
            image_1 = batch['image1_data'][minibatch].to(device=device, dtype=dtype)
            image_1 = image_1.view(1,1,256,256,256)

            label_1 = batch['image1_label'][minibatch].to(device=device, dtype=dtype)
            label_1 = label_1.view(1,3,256,256,256)

            bv_label = label_1[:, 2, :, :, :]
            bv_label = bv_label.view(1,1,256,256,256)
            
            del label_1

            original_res = [a[minibatch].item() for a in batch['original_resolution']]

            image_1_resize = F.interpolate(image_1, size=original_res, mode='trilinear', align_corners=True)
            image_1_resize = image_1_resize.view(1,1,original_res[0], original_res[1], original_res[2])
            
            bv_label_resize = F.interpolate(bv_label, size=original_res, mode='trilinear', align_corners=True)
            
            del bv_label
            
            # Get coarse output from deeplab model from 256 resolution input
            out_coarse = deeplab(image_1)
            out_coarse = out_coarse.view(1,3,256,256,256)
            
            del image_1

            bv_coarse = out_coarse[:, 2, :, :, :]
            bv_coarse = bv_coarse.view(1,1,256,256,256)
            
            del out_coarse
        
            bv_coarse_resize = F.interpolate(bv_coarse, size=original_res, mode='trilinear', align_corners=True)
            
            del bv_coarse
            
            box_size = 192
            half_size = int(box_size / 2)
            
            image_size_x = int(image_1_resize.shape[-3])
            image_size_y = int(image_1_resize.shape[-2])
            image_size_z = int(image_1_resize.shape[-1])
            
            x,y,z = loadbvcenter(binarize_output(bv_coarse_resize).view([1] + original_res))
            x, y, z = np.clip([x, y, z], a_min=box_size-half_size, a_max=box_size+half_size)
            x1 = max(x-half_size, 0)
            x2 = min(x+half_size, image_size_x)
            y1 = max(y-half_size, 0)
            y2 = min(y+half_size, image_size_y)
            z1 = max(z-half_size, 0)
            z2 = min(z+half_size, image_size_z)
            
            
            #bbox_bv = bv_coarse_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            #bbox_bv = reshape_image(bbox_bv.squeeze(), box_size, box_size, box_size).to(device, dtype)
            #bbox_bv = bbox_bv.view(1,1,box_size,box_size,box_size)
            
            del bv_coarse_resize
            
            bbox_bv_label = bv_label_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
            bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_bv_label = bbox_bv_label.view(1,1,box_size,box_size,box_size)
            
            del bv_label_resize
            
            #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
            bbox_image = image_1_resize[:, :, x1:x2, y1:y2, z1:z2]
            bbox_image = reshape_image(bbox_image.squeeze(), box_size, box_size, box_size).to(device, dtype)
            bbox_image = bbox_image.view(1, 1, box_size, box_size, box_size)
            
            del image_1_resize
            
            #bbox_iamge, bbox_bv_label, bbox_bv = get_bboxes(image_1_resize, bv_label_resize, bv_coarse_resize, 1, 200)
            
            #bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)
            
            refine_out = refine_model(bbox_image)
            
            del bbox_image
            
            #refine_out = refine_model(seg_image_concat)
            # do the inference

            #print(refine_out.shape)
            #print(bbox_bv_label.shape)

            loss = dice_loss(refine_out, bbox_bv_label)
            print(loss.item())
            train_losses.append(loss)
            
            torch.cuda.empty_cache()
        
        loss = sum(train_losses) / BATCH_SIZE
        train_losses=[]
        epoch_loss += loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        #scheduler.step(loss_1)
        # take a gradient descent step
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%1 == 0:
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
            # iter over validation mini batches
                val_losses = []
                for minibatch in range(BATCH_SIZE):
                    
                    image_1 = batch['image1_data'][minibatch].to(device=device, dtype=dtype)
                    image_1 = image_1.view(1,1,256,256,256)

                    label_1 = batch['image1_label'][minibatch].to(device=device, dtype=dtype)
                    label_1 = label_1.view(1,3,256,256,256)

                    bv_label = label_1[:, 2, :, :, :]
                    bv_label = bv_label.view(1,1,256,256,256)

                    original_res = [a[minibatch].item() for a in batch['original_resolution']]

                    image_1_resize = F.interpolate(image_1, size=original_res, mode='trilinear', align_corners=True)
                    image_1_resize = image_1_resize.view(1,1,original_res[0], original_res[1], original_res[2])

                    bv_label_resize = F.interpolate(bv_label, size=original_res, mode='trilinear', align_corners=True)

                    # Get coarse output from deeplab model from 256 resolution input
                    out_coarse = deeplab(image_1)
                    out_coarse = out_coarse.view(1,3,256,256,256)

                    bv_coarse = out_coarse[:, 2, :, :, :]
                    bv_coarse = bv_coarse.view(1,1,256,256,256)

                    bv_coarse_resize = F.interpolate(bv_coarse, size=original_res, mode='trilinear', align_corners=True)

                    box_size = 192
                    half_size = int(box_size / 2)

                    image_size_x = int(image_1_resize.shape[-3])
                    image_size_y = int(image_1_resize.shape[-2])
                    image_size_z = int(image_1_resize.shape[-1])

                    x,y,z = loadbvcenter(binarize_output(bv_coarse_resize).view([1] + original_res))
                    x, y, z = np.clip([x, y, z], a_min=box_size-half_size, a_max=box_size+half_size)
                    x1 = max(x-half_size, 0)
                    x2 = min(x+half_size, image_size_x)
                    y1 = max(y-half_size, 0)
                    y2 = min(y+half_size, image_size_y)
                    z1 = max(z-half_size, 0)
                    z2 = min(z+half_size, image_size_z)


                    bbox_bv = bv_coarse_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
                    bbox_bv = reshape_image(bbox_bv.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_bv = bbox_bv.view(1,1,box_size,box_size,box_size)

                    bbox_bv_label = bv_label_resize.view(original_res)[x1:x2, y1:y2, z1:z2]
                    bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_bv_label = bbox_bv_label.view(1,1,box_size,box_size,box_size)

                    #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
                    bbox_image = image_1_resize[:, :, x1:x2, y1:y2, z1:z2]
                    bbox_image = reshape_image(bbox_image.squeeze(), box_size, box_size, box_size).to(device, dtype)
                    bbox_image = bbox_image.view(1, 1, box_size, box_size, box_size)

                    #bbox_iamge, bbox_bv_label, bbox_bv = get_bboxes(image_1_resize, bv_label_resize, bv_coarse_resize, 1, 200)

                    #bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)

                    refine_out = refine_model(bbox_image)
                    #refine_out = refine_model(seg_image_concat)
                    # do the inference

                    #print(refine_out.shape)
                    #print(bbox_bv_label.shape)

                    loss = dice_loss(refine_out, bbox_bv_label)
                    val_losses.append(loss)
                
                avg_loss = sum(val_losses) / BATCH_SIZE
                val_losses = []
                print(avg_loss)
            
                # calculate loss
                valloss_1 += avg_loss.item()
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            #scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                save_1('refine_bv_vnet2_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv_vnet2_save', refine_model, optimizer, logger, e, scheduler)

            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

0.9914483428001404
0.995200514793396
0.988614022731781
0.9950375556945801
0.9892377257347107
0.990784764289856
0.9860950708389282
0.9988768100738525
0.9981459379196167
0.985699474811554
0.9904349446296692
0.9857410788536072
0.9883083701133728
0.9904637932777405
0.9917328357696533
0.9867354035377502
0.9855008721351624
0.9937041401863098
0.9906130433082581
0.9867554306983948
0.9872675538063049
0.9859113693237305
0.9845146536827087
0.9828760027885437
0.9836624264717102
0.9896026253700256
0.9817561507225037
0.9823893308639526
0.992265522480011
0.9891906380653381
0.9957242012023926
0.9900792241096497
0.9860095381736755
0.969079852104187
0.9881563186645508
0.9861535429954529
0.9875392317771912
0.9780211448669434
0.9913449287414551
0.9747231602668762
0.9748871326446533
0.9869660139083862
0.9788895845413208
0.9810425639152527
0.9749435782432556
0.9856597185134888
0.9764523506164551
0.9665570855140686
0.9721750020980835
0.9700875282287598
0.9682819247245789
0.9520896077156067
0.9800772666931152

  0%|          | 1/4999 [17:35<1465:41:00, 1055.71s/it]

Checkpoint 1 saved !
------- 1st valloss=0.9747

0.9751170873641968
0.998724102973938
0.9740279316902161
0.9877938628196716
0.9335167407989502
0.7912269830703735
0.770664632320404
0.9412925839424133
0.9956787824630737
0.8347064256668091
0.6594169735908508
0.5650671720504761
0.8651669025421143
0.9972668886184692
0.9981608986854553
0.9860399961471558
0.9638130068778992
0.9964104890823364
0.9753457903862
0.9911590814590454
0.9881216883659363
0.9273332953453064
0.9952103495597839
0.9919453263282776
0.9883142709732056
0.8966168165206909
0.928007960319519
0.9960810542106628
0.9976407885551453
0.8779322504997253
0.9155575037002563
0.7877359390258789
0.9696472883224487
0.9957005381584167
0.9986253380775452
0.7839053273200989
0.8772719502449036
0.9159700870513916
0.8705358505249023
0.7306325435638428
0.7601844072341919
0.9966649413108826
0.9483731389045715
0.9978011846542358
0.9971268177032471
0.961749792098999
0.8235249519348145
0.7335331439971924
0.7437876462936401
0.7459753751754761
0.945480

  0%|          | 2/4999 [33:45<1429:44:11, 1030.03s/it]

...

Checkpoint 136 saved !
------- 1st valloss=0.0452

0.05087536573410034
0.07531124353408813
0.33429789543151855
0.034075260162353516
0.13220125436782837
0.05113780498504639
0.09787231683731079
0.09482282400131226
0.5444990396499634
0.12523925304412842
0.25942015647888184
0.050341665744781494
0.1412644386291504
0.050998032093048096
0.1530638337135315
0.06804019212722778
0.3283613920211792
0.12917011976242065
0.11603319644927979
0.059590041637420654
0.07740062475204468
0.16613703966140747
0.5809224843978882
0.18648642301559448
0.11186379194259644
0.059487342834472656
0.29685133695602417
0.06642812490463257
0.11588740348815918
0.04197680950164795
0.13695496320724487
0.43691474199295044
0.10740786790847778
0.06928521394729614
0.3131633996963501
0.395791232585907
0.1745484471321106
0.174699604511261
0.07384532690048218
0.08773607015609741
0.20312762260437012
0.05222600698471069
0.03858691453933716
0.14711976051330566
0.05400055646896362
0.07673722505569458
0.09694463014602661
0.0848723649978

  3%|▎         | 137/4999 [36:58:07<1308:38:19, 968.96s/it]

Checkpoint 137 saved !
------- 1st valloss=0.9988

0.04485654830932617
0.09415262937545776
0.06891071796417236
0.06837165355682373
0.06091958284378052
0.11061948537826538
0.06557011604309082
0.32182854413986206
0.13066667318344116
0.04855644702911377
0.08774876594543457
0.14232826232910156
0.0867159366607666
0.09236305952072144
0.07573843002319336
0.140394926071167
0.24701637029647827
0.1699545979499817
0.05327045917510986
0.3316482901573181
0.10360848903656006
0.09280163049697876
0.1758638620376587
0.12352412939071655
0.12874239683151245
0.2058916687965393
0.05557084083557129
0.0793309211730957
0.050213754177093506
0.03723776340484619
0.06265610456466675
0.0398716926574707
0.14182060956954956
0.2088741660118103
0.10598719120025635
0.18634164333343506
0.10020732879638672
0.059018731117248535
0.05131798982620239
0.053939104080200195
0.14041048288345337
0.07276690006256104
0.04709315299987793
0.08208990097045898
0.2095646858215332
0.19064486026763916
0.06139731407165527
0.169247984886169

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.13671177625656128
0.09286803007125854
0.04016757011413574
0.1874421238899231
0.07998967170715332
0.14210641384124756
0.05935412645339966
0.08048170804977417
0.06587713956832886
0.12266701459884644
0.1224026083946228
0.12364864349365234
0.11284351348876953
0.09697741270065308
0.05843818187713623
0.06258612871170044
0.21359556913375854
0.06596213579177856
0.10985487699508667
0.09345024824142456
0.11223536729812622
0.059485554695129395
0.1202649474143982
0.43970900774002075
0.18632274866104126
0.058908045291900635
0.13028526306152344
0.1272013783454895
0.050528645515441895
0.09981763362884521
0.07405680418014526
0.1455807089805603
0.05819135904312134
0.059735894203186035
0.09108740091323853
0.041212260723114014
0.10423588752746582
0.21713018417358398
0.07904839515686035
0.04692763090133667
0.11985671520233154
0.12072980403900146
0.07785552740097046
0.07618552446365356
0.15138018131256104
0.04568904638290405
0.38025641441345215
0.10279935598373413
0.07615059614181519
0.07705694437026978


In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()
        
        if bv.item() >= 0.2 or bd.item() >= 0.1:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)